In [ ]:
import sys
import gpytorch
import numpy as np
import pandas as pd
import torch
from datetime import datetime
from scipy.stats import qmc
import itertools
from itertools import combinations_with_replacement, combinations, permutations

import bo_methods_lib
from bo_methods_lib.bo_methods_lib.GPBO_Classes_New import * #Fix this later
from bo_methods_lib.bo_methods_lib.GPBO_Class_fxns import * #Fix this later
from bo_methods_lib.bo_methods_lib.analyze_data import * #Fix this later
from bo_methods_lib.bo_methods_lib.GPBO_Classes_plotters import * #Fix this later
import pympler
import pickle
import signac

from pympler import asizeof

from matplotlib import pyplot as plt

import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 200

#Ignore inconcistent version warning
import warnings
# from sklearn.exceptions import InconsistentVersionWarning
# warnings.filterwarnings(action='ignore', category=InconsistentVersionWarning)
warnings.filterwarnings('ignore')

In [ ]:
# Define small case study
cs_val1 = 1
cs_val2 = 2
num_x_data = 5
gen_meth_x = Gen_meth_enum(1)
num_theta_data1 = 10
num_theta_data2 = 5
gen_meth_theta = Gen_meth_enum(1)

ep0 = 1
sep_fact = 0.8
normalize = True
noise_mean = 0
noise_std = 0.01
kernel = Kernel_enum(1)
lenscl = 1
outputscl = 1
retrain_GP = 0
seed = 1
method = GPBO_Methods(Method_name_enum(3))  # 2A

# Define cs_params, simulator, and exp_data for CS1
simulator1 = simulator_helper_test_fxns(cs_val2, noise_mean, noise_std, seed)
exp_data1 = simulator1.gen_exp_data(num_x_data, gen_meth_x)
sim_data1 = simulator1.gen_sim_data(
    num_theta_data1, num_x_data, gen_meth_theta, gen_meth_x, sep_fact
)
val_data1 = simulator1.gen_sim_data(
    num_theta_data1, num_x_data, gen_meth_theta, gen_meth_x, sep_fact, True
)
gp_emulator1_e = Type_2_GP_Emulator(
    sim_data1,
    val_data1,
    None,
    None,
    None,
    kernel,
    lenscl,
    noise_std,
    outputscl,
    retrain_GP,
    seed,
    normalize,
    None,
    None,
    None,
    None,
)

# Define cs_params, simulator, and exp_data for CS2
simulator2 = simulator_helper_test_fxns(cs_val2, noise_mean, noise_std, seed)
exp_data2 = simulator2.gen_exp_data(num_x_data, gen_meth_x)
sim_data2 = simulator2.gen_sim_data(
    num_theta_data2, num_x_data, gen_meth_theta, gen_meth_x, sep_fact
)
val_data2 = simulator2.gen_sim_data(
    num_theta_data2, num_x_data, gen_meth_theta, gen_meth_x, sep_fact, True
)
gp_emulator2_e = Type_2_GP_Emulator(
    sim_data2,
    val_data2,
    None,
    None,
    None,
    kernel,
    lenscl,
    noise_std,
    outputscl,
    retrain_GP,
    seed,
    normalize,
    None,
    None,
    None,
    None,
)

covar = False
gp_emulator = gp_emulator2_e
exp_data = exp_data2
simulator = simulator2

train_data, test_data = gp_emulator.set_train_test_data(sep_fact, seed)
gp_emulator.train_gp()
gp_mean, gp_var = gp_emulator.eval_gp_mean_var_val(False) #Calc mean, var of gp
gp_mean, gp_covar = gp_emulator.eval_gp_mean_var_val(True) #Calc mean, var of gp

print(gp_mean)
print(gp_var)
print(gp_mean.shape)
print(gp_var.shape)

In [ ]:
#Set Date and Time
dateTimeObj = datetime.now()
timestampStr = dateTimeObj.strftime("%d-%b-%Y (%H:%M:%S)")
# print("Date and Time: ", timestampStr)
# DateTime = dateTimeObj.strftime("%Y/%m/%d/%H-%M-%S%p")
DateTime = dateTimeObj.strftime("%Y/%m/%d/%H-%M")
DateTime = None ##For Testing

cs_name1  = CS_name_enum(1)
cs_name2  = CS_name_enum(2)

# #Define small case study
# num_x_data = 5
# gen_meth_x = Gen_meth_enum(2)
# num_theta_data1 = 20
# num_theta_data2 = 20
# gen_meth_theta = Gen_meth_enum(1)

# ep0 = 1
# sep_fact = 0.8
# normalize = True
# noise_mean = 0
# noise_std = 0.01
# kernel = Kernel_enum(1)
# lenscl = None
# outputscl = None
# retrain_GP = 1
# seed = 1
# method = GPBO_Methods(Method_name_enum(1)) #1A

#Define cs_params, simulator, and exp_data for CS1
# simulator1 = simulator_helper_test_fxns(cs_name1.value, noise_mean, noise_std, seed)
# exp_data1 = simulator1.gen_exp_data(num_x_data, gen_meth_x)
# sim_data1 = simulator1.gen_sim_data(num_theta_data1, num_x_data, gen_meth_theta, gen_meth_x, sep_fact)
# sim_sse_data1 = simulator1.sim_data_to_sse_sim_data(method, sim_data1, exp_data1, sep_fact)
# val_data1 = simulator1.gen_sim_data(num_theta_data1, num_x_data, gen_meth_theta, gen_meth_x, sep_fact, True)
# val_sse_data1 = simulator1.sim_data_to_sse_sim_data(method, val_data1, exp_data1, sep_fact, True)
# gp_emulator1_s = Type_1_GP_Emulator(sim_sse_data1, val_sse_data1, None, None, None, kernel, lenscl, noise_std, outputscl, retrain_GP, seed, normalize, None, None, None, None)

# #Define cs_params, simulator, and exp_data for CS2
# simulator2 = simulator_helper_test_fxns(cs_name2.value, noise_mean, noise_std, seed)
# exp_data2 = simulator2.gen_exp_data(num_x_data, gen_meth_x)
# sim_data2 = simulator2.gen_sim_data(num_theta_data2, num_x_data, gen_meth_theta, gen_meth_x, sep_fact)
# sim_sse_data2 = simulator2.sim_data_to_sse_sim_data(method, sim_data2, exp_data2, sep_fact)
# val_data2 = simulator2.gen_sim_data(num_theta_data2, num_x_data, gen_meth_theta, gen_meth_x, sep_fact, True)
# val_sse_data2 = simulator2.sim_data_to_sse_sim_data(method, val_data2, exp_data2, sep_fact, True)
# gp_emulator2_s = Type_1_GP_Emulator(sim_sse_data2, val_sse_data2, None, None, None, kernel, lenscl, noise_std, outputscl, retrain_GP, seed, normalize, None, None, None, None)

#Define small case study
num_x_data = 5
gen_meth_x = Gen_meth_enum(2)
num_theta_data1 = 10
num_theta_data2 = 5
gen_meth_theta = Gen_meth_enum(1)

ep0 = 1
sep_fact = 0.8
normalize = True
noise_mean = 0
noise_std = 0.01
kernel = Kernel_enum(1)
lenscl = 1
outputscl = 1
retrain_GP = 0
seed = 1
method = GPBO_Methods(Method_name_enum(5)) #2C

#Define cs_params, simulator, and exp_data for CS1
simulator1 = simulator_helper_test_fxns(cs_name1.value, noise_mean, noise_std, seed)
exp_data1 = simulator1.gen_exp_data(num_x_data, gen_meth_x)
sim_data1 = simulator1.gen_sim_data(num_theta_data1, num_x_data, gen_meth_theta, gen_meth_x, sep_fact)
val_data1 = simulator1.gen_sim_data(num_theta_data1, num_x_data, gen_meth_theta, gen_meth_x, sep_fact, True)
gp_emulator1_e = Type_2_GP_Emulator(sim_data1, val_data1, None, None, None, kernel, lenscl, noise_std, outputscl, retrain_GP, seed, normalize, None, None, None, None)

#Define cs_params, simulator, and exp_data for CS2
simulator2 = simulator_helper_test_fxns(cs_name2.value, noise_mean, noise_std, seed)
exp_data2 = simulator2.gen_exp_data(num_x_data, gen_meth_x)
sim_data2 = simulator2.gen_sim_data(num_theta_data2, num_x_data, gen_meth_theta, gen_meth_x, sep_fact)
val_data2 = simulator2.gen_sim_data(num_theta_data2, num_x_data, gen_meth_theta, gen_meth_x, sep_fact, True)
gp_emulator2_e = Type_2_GP_Emulator(sim_data2, val_data2, None, None, None, kernel, lenscl, noise_std, outputscl, retrain_GP, seed, normalize, None, None, None, None)

gp_emulator = gp_emulator1_e
exp_data = exp_data1
simulator = simulator1
covar = False
train_data, test_data = gp_emulator.set_train_test_data(sep_fact, seed)
gp_emulator.train_gp()
misc_data = Data(None, exp_data.x_vals, None, None,None,None,None,None, simulator.bounds_theta_reg, simulator.bounds_x, sep_fact, seed)
theta = gp_emulator.gp_val_data.theta_vals[0].reshape(1,-1) #Set "candidate thetas"
theta_vals = np.repeat(theta.reshape(1,-1), exp_data.get_num_x_vals() , axis =0)
misc_data.theta_vals = theta_vals #Set misc thetas
feature_misc_data = gp_emulator.featurize_data(misc_data) #Set feature vals
misc_data.gp_mean, misc_data.gp_var = gp_emulator.eval_gp_mean_var_misc(misc_data, feature_misc_data) #Calc mean, var of gp 
sse_mean, sse_var = gp_emulator.eval_gp_sse_var_misc(misc_data, method, exp_data, covar) #Calc mean, var of gp sse
print(sse_mean)
print(sse_var)